<img src="https://github.com/hernancontigiani/ceia_memorias_especializacion/raw/master/Figures/logoFIUBA.jpg" width="500" align="center">




# Procesamiento de lenguaje natural



## Custom embedddings con Gensim



### Objetivo
El objetivo es utilizar documentos / corpus para crear embeddings de palabras basado en ese contexto. Se utilizará canciones de bandas para generar los embeddings, es decir, que los vectores tendrán la forma en función de como esa banda haya utilizado las palabras en sus canciones.

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import multiprocessing
from gensim.models import Word2Vec

### Datos
Utilizaremos como dataset canciones de bandas de habla inglesa.

In [4]:
import zipfile
# Descargar la carpeta de dataset
import os
import platform
if os.access('./songs_dataset', os.F_OK) is False:
    if os.access('songs_dataset.zip', os.F_OK) is False:
        if platform.system() == 'Windows':
            !curl https://raw.githubusercontent.com/FIUBA-Posgrado-Inteligencia-Artificial/procesamiento_lenguaje_natural/main/datasets/songs_dataset.zip -o songs_dataset.zip
        else:
            !wget songs_dataset.zip https://github.com/FIUBA-Posgrado-Inteligencia-Artificial/procesamiento_lenguaje_natural/raw/main/datasets/songs_dataset.zip
    #!unzip -q songs_dataset.zip   
    with zipfile.ZipFile('songs_dataset.zip', 'r') as zip_ref:
        zip_ref.extractall('./')
else:
    print("El dataset ya se encuentra descargado")

In [ ]:
# Posibles bandas
os.listdir("./songs_dataset/")

In [ ]:
# Armar el dataset utilizando salto de línea para separar las oraciones/docs
df = pd.read_csv('songs_dataset/beatles.txt', sep='/n', header=None)
df.head()

In [ ]:
print("Cantidad de documentos:", df.shape[0])

### 1 - Preprocesamiento

In [17]:
from tensorflow.keras.preprocessing.text import text_to_word_sequence

sentence_tokens = []
# Recorrer todas las filas y transformar las oraciones
# en una secuencia de palabras (esto podría realizarse con NLTK o spaCy también)
for _, row in df[:None].iterrows():
    sentence_tokens.append(text_to_word_sequence(row[0]))

In [ ]:
# Demos un vistazo
sentence_tokens[:2]

### 2 - Crear los vectores (word2vec)

In [35]:
from gensim.models.callbacks import CallbackAny2Vec
# Durante el entrenamiento gensim por defecto no informa el "loss" en cada época
# Sobrecargamos el callback para poder tener esta información
class callback(CallbackAny2Vec):
    """
    Callback to print loss after each epoch
    """
    def __init__(self):
        self.epoch = 0

    def on_epoch_end(self, model):
        loss = model.get_latest_training_loss()
        if self.epoch == 0:
            print('Loss after epoch {}: {}'.format(self.epoch, loss))
        else:
            print('Loss after epoch {}: {}'.format(self.epoch, loss- self.loss_previous_step))
        self.epoch += 1
        self.loss_previous_step = loss

In [36]:
# Crearmos el modelo generador de vectores
# En este caso utilizaremos la estructura modelo Skipgram
w2v_model = Word2Vec(min_count=5,    # frecuencia mínima de palabra para incluirla en el vocabulario
                     window=2,       # cant de palabras antes y desp de la predicha
                     vector_size=300,       # dimensionalidad de los vectores 
                     negative=20,    # cantidad de negative samples... 0 es no se usa
                     workers=1,      # si tienen más cores pueden cambiar este valor
                     sg=1)           # modelo 0:CBOW  1:skipgram

In [37]:
# Obtener el vocabulario con los tokens
w2v_model.build_vocab(sentence_tokens)

In [ ]:
# Cantidad de filas/docs encontradas en el corpus
print("Cantidad de docs en el corpus:", w2v_model.corpus_count)

In [ ]:
# Cantidad de words encontradas en el corpus
print("Cantidad de words distintas en el corpus:", len(w2v_model.wv.index_to_key))

### 3 - Entrenar embeddings

In [ ]:
# Entrenamos el modelo generador de vectores
# Utilizamos nuestro callback
w2v_model.train(sentence_tokens,
                 total_examples=w2v_model.corpus_count,
                 epochs=100,
                 compute_loss = True,
                 callbacks=[callback()]
                 )

### 4 - Ensayar

In [ ]:
# Palabras que MÁS se relacionan con...:
w2v_model.wv.most_similar(positive=["darling"], topn=10)

In [ ]:
# Palabras que MENOS se relacionan con...:
w2v_model.wv.most_similar(negative=["love"], topn=10)

In [ ]:
# Palabras que MÁS se relacionan con...:
w2v_model.wv.most_similar(positive=["four"], topn=10)

In [ ]:
# Palabras que MÁS se relacionan con...:
w2v_model.wv.most_similar(positive=["money"], topn=5)

In [ ]:
# Ensayar con una palabra que no está en el vocabulario:
w2v_model.wv.most_similar(negative=["diedaa"])

In [ ]:
# el método `get_vector` permite obtener los vectores:
vector_love = w2v_model.wv.get_vector("love")
print(vector_love)

In [ ]:
# el método `most_similar` también permite comparar a partir de vectores
w2v_model.wv.most_similar(vector_love)

In [ ]:
# Palabras que MÁS se relacionan con...:
w2v_model.wv.most_similar(positive=["love"], topn=10)

### 5 - Visualizar agrupación de vectores

In [49]:
from sklearn.decomposition import IncrementalPCA    
from sklearn.manifold import TSNE                   
import numpy as np                                  

def reduce_dimensions(model, num_dimensions = 2 ):
     
    vectors = np.asarray(model.wv.vectors)
    labels = np.asarray(model.wv.index_to_key)  

    tsne = TSNE(n_components=num_dimensions, random_state=0)
    vectors = tsne.fit_transform(vectors)

    return vectors, labels

In [ ]:
# Graficar los embedddings en 2D
import plotly.graph_objects as go
import plotly.express as px

vecs, labels = reduce_dimensions(w2v_model)

MAX_WORDS=200
fig = px.scatter(x=vecs[:MAX_WORDS,0], y=vecs[:MAX_WORDS,1], text=labels[:MAX_WORDS])
fig.show()
#fig.show(renderer="colab") # esto para plotly en colab

In [ ]:
# Graficar los embedddings en 3D

vecs, labels = reduce_dimensions(w2v_model,3)

fig = px.scatter_3d(x=vecs[:MAX_WORDS,0], y=vecs[:MAX_WORDS,1], z=vecs[:MAX_WORDS,2],text=labels[:MAX_WORDS])
fig.update_traces(marker_size = 2)
#fig.show(renderer="colab") # esto para plotly en colab

In [56]:
# También se pueden guardar los vectores y labels como tsv para graficar en
# http://projector.tensorflow.org/


vectors = np.asarray(w2v_model.wv.vectors)
labels = list(w2v_model.wv.index_to_key)

np.savetxt("vectors.tsv", vectors, delimiter="\t")

with open("labels.tsv", "w") as fp:
    for item in labels:
        fp.write("%s\n" % item)

### Alumno

- Crear sus propios vectores con Gensim basado en lo visto en clase con otro dataset.
- Probar términos de interés y explicar similitudes en el espacio de embeddings (sacar conclusiones entre palabras similitudes y diferencias).
- Graficarlos.
- Obtener conclusiones.

# Solución propuesta

El dataset seleccionado es un libro en ingles llamado "Odyssey of the Dragonlords". Este libro es una aventura prediseñada para Dungeons Masters y la version 5.0 de Dungeons & Dragons. El documento original está en formato .pdf y se utilizaron herramientas online para transformarlo a .txt.

A grandes rasgos, este libro desarrolla el camino que emprende un grupo de aventureros en un entorno similar a la mitologia griega (con dioses e historia específicos del setting del libro) para salvar a su mundo de la perdición, y cómo ellos logran ascender a la divinidad en el proceso.

Como el libro original esta en dos columnas, gran parte del texto esta separado y se requiere una libreria adicional para facilitar la tokenizacion de las oraciones (nltk). Adicionalmente, en caso de un libro (que no ocurre en canciones normalmente), se debe separar oraciones no solo por puntos, sino tambien por signos de interrogracion y exclamacion. 

Algunas desventajas de este metodo:

 - Nombres como por ejemplo Michal E. Cross separan oraciones
 - La conversion de pdf a texto no es perfecta. Aparecen algunos artefactos como palabras separadas (_T he_ en vez de _The_)

In [29]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
import seaborn as sns
import multiprocessing
from gensim.models import Word2Vec
import zipfile
import os
import platform
import plotly.graph_objects as go
import plotly.express as px
from tensorflow.keras.preprocessing.text import text_to_word_sequence
from gensim.models.callbacks import CallbackAny2Vec
from sklearn.decomposition import IncrementalPCA    
from sklearn.manifold import TSNE
import re
import nltk
nltk.download('punkt')
from nltk.tokenize.punkt import PunktSentenceTokenizer, PunktParameters 

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Nicolas\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
punkt_param = PunktParameters()
punkt_param.abbrev_types = set()
sentence_tokenizer = PunktSentenceTokenizer(punkt_param)

def custom_tokenize(text):
    # Split text on the specified punctuation marks
    sentences = re.split(r'(?<!\w\.\w.)(?<![A-Z][a-z]\.)(?<=\.|\?|!)\s', text)
    return sentences


with open("Odyssey of the Dragonlords.txt", "r") as file:
    lines = file.readlines()
    text = " ".join(line.strip() for line in lines)
    
sentences = custom_tokenize(text)

# Create a DataFrame from the sentences
df = pd.DataFrame(sentences, columns=['Sentence'])

df.head(10)

,Sentence
0,Welcome to Odyssey of the Dragonlords.
1,This booklet will provide you with everything ...
2,Odyssey of the Dragonlords is heavily inspired...
3,"As we designed the campaign, we sought to incl..."
4,"However, Thylea is not ancient Greece."
5,"You will not find Zeus, Athena, or Apollo amon..."
6,"As you explore Thylea, you will encounter fami..."
7,Mortals have only recently come to these lands.
8,The world of Thylea blends high fantasy with t...
9,"Elves, dwarves, and halflings now live alongsi..."


In [3]:
print("Cantidad de documentos:", df.shape[0])

Cantidad de documentos: 17618


In [4]:
sentence_tokens = []

for _, row in df[:None].iterrows():
    sentence_tokens.append(text_to_word_sequence(row[0]))

sentence_tokens[8783]

C:\Users\Nicolas\AppData\Local\Temp\ipykernel_13728\4086292549.py:4: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  sentence_tokens.append(text_to_word_sequence(row[0]))


['a',
 'fearsome',
 'minotaur',
 'and',
 'a',
 'massive',
 'gygan',
 'warrior',
 'are',
 'fighting',
 'each',
 'other',
 'honing',
 'their',
 'skills',
 'for',
 'battle']

In [5]:
from gensim.models.callbacks import CallbackAny2Vec
# Durante el entrenamiento gensim por defecto no informa el "loss" en cada época
# Sobrecargamos el callback para poder tener esta información
class callback(CallbackAny2Vec):
    """
    Callback to print loss after each epoch
    """
    def __init__(self):
        self.epoch = 0

    def on_epoch_end(self, model):
        loss = model.get_latest_training_loss()
        if self.epoch == 0:
            print('Loss after epoch {}: {}'.format(self.epoch, loss))
        else:
            print('Loss after epoch {}: {}'.format(self.epoch, loss- self.loss_previous_step))
        self.epoch += 1
        self.loss_previous_step = loss


In [6]:
# Crearmos el modelo generador de vectores
# En este caso utilizaremos la estructura modelo Skipgram
w2v_model = Word2Vec(min_count=5,    # frecuencia mínima de palabra para incluirla en el vocabulario
                     window=5,       # cant de palabras antes y desp de la predicha
                     vector_size=1000,       # dimensionalidad de los vectores 
                     negative=20,    # cantidad de negative samples... 0 es no se usa
                     workers=12,      # si tienen más cores pueden cambiar este valor
                     sg=1)           # modelo 0:CBOW  1:skipgram

In [7]:
# Obtener el vocabulario con los tokens
w2v_model.build_vocab(sentence_tokens)

In [8]:
# Cantidad de filas/docs encontradas en el corpus
print("Cantidad de docs en el corpus:", w2v_model.corpus_count)
# Cantidad de words encontradas en el corpus
print("Cantidad de words distintas en el corpus:", len(w2v_model.wv.index_to_key))

Cantidad de docs en el corpus: 17618
Cantidad de words distintas en el corpus: 5454


In [9]:
train = False
if (train):
    # Entrenamos el modelo generador de vectores
    # Utilizamos nuestro callback
    w2v_model.train(sentence_tokens,
                    total_examples=w2v_model.corpus_count,
                    epochs=100,
                    compute_loss = True,
                    callbacks=[callback()]
                    )
    # Guardar el modelo
    w2v_model.save("w2v_model.model")
else:
    w2v_model = Word2Vec.load("w2v_model.model")

## Relaciones de palabras

### Lutheria

In [10]:
# Palabras que MÁS se relacionan con...:
w2v_model.wv.most_similar(positive=["lutheria"], topn=10)

[('twin', 0.32119855284690857),
 ('dominion', 0.29322871565818787),
 ('mistress', 0.29278141260147095),
 ('humor', 0.29062914848327637),
 ('m20', 0.29052576422691345),
 ('sydon', 0.2856642007827759),
 ('confronting', 0.2844409644603729),
 ('scythe', 0.27126264572143555),
 ('pellenia', 0.2703191041946411),
 ('ascended', 0.2695517838001251)]

Lutheria, junto con Sydon, son los antagonistas principales de la historia, en la que ellos son mellizos. Sin embargo, en la aventura Sydon juega un rol mas activo, y por ende se suele llamar a Lutheria mas por su rol como melliza (twin) que como su propia persona. Su arma caracteristica es una guadaña (scythe). El lugar donde los aventureros se la encuentran para enfrentarla esta denotado como M20. Pellenia es un personaje a quien Lutheria capturó durante la guerra, y logró escaparse acoplandose al humor oscuro de Lutheria.

### Sydon

In [11]:
w2v_model.wv.most_similar(positive=["sydon"], topn=10)

[('demanded', 0.3137003779411316),
 ('visage', 0.30404555797576904),
 ('lutheria', 0.28566423058509827),
 ('prayer', 0.2799292504787445),
 ('obedience', 0.2785988748073578),
 ('enslaved', 0.27811679244041443),
 ('chalcia', 0.27806198596954346),
 ('valued', 0.2769514322280884),
 ('goloron', 0.2692423462867737),
 ('letters', 0.2684606909751892)]

Sydon, como antagonista principal de la historia, se comunica con sus vasallos por medio de exigencias (demanded, prayer, obedience, enslaved). Tiene varios hijos, como Goloron y Chalcia.

### Dragonlord

In [34]:
w2v_model.wv.most_similar(positive=["dragonlord"], topn=10)

[('—rizon', 0.40449315309524536),
 ('phobas', 0.3691289722919464),
 ('rizon', 0.36301249265670776),
 ('huorath', 0.3129887282848358),
 ('da', 0.30400747060775757),
 ('options', 0.29541727900505066),
 ('inscription', 0.28474023938179016),
 ('telamok', 0.28195810317993164),
 ('wyrmling', 0.2817297875881195),
 ('dr', 0.28172892332077026)]

En la aventura, hace 500 años el mundo peligraba mucho por la furia de los Dioses antagonistas. Ese yugo fue destruido por 5 Dragonlords (Rizon Phobas siendo dos de ellos). Actualmente sus cuerpos yacen en Telamok, una necropolis.

### Telamok

In [35]:
w2v_model.wv.most_similar(positive=["telamok"], topn=10)

[('necropolis', 0.5355798006057739),
 ('isadore', 0.42447105050086975),
 ('recognizes', 0.400800496339798),
 ('ochos', 0.3859226703643799),
 ('oathsworn', 0.3764425814151764),
 ('savage', 0.3762202560901642),
 ('interred', 0.3749942183494568),
 ('damon', 0.37309616804122925),
 ('arkelander', 0.3687095046043396),
 ('maintained', 0.3405444920063019)]

Telamok es una necropolis en la cual yacen los cuerpos de los Dragonlords y sus descendientes (Ochos Arkelander y Isadore Arkelander). La custodia y mantiene Damon.

### Arkelander

In [36]:
w2v_model.wv.most_similar(positive=["arkelander"], topn=10)

[('estor', 0.6508475542068481),
 ('ochos', 0.4653950035572052),
 ('atrocities', 0.4040852189064026),
 ('lysis', 0.39857515692710876),
 ('savage', 0.3852740228176117),
 ('ghost', 0.3820172846317291),
 ('mast', 0.3796650469303131),
 ('telamok', 0.3687095046043396),
 ('descendant', 0.3686769902706146),
 ('bloodthirsty', 0.36141887307167053)]

Arkelander es el apellido de Estor, el capitan de los Dragonlords. Ochos y Lysis con descendientes de el. Traicionó a sus compañeros, cometiendo atrocidades (atrocities). Ronda el mundo como un fantasma (ghost) con sed de venganza (bloodthirsty).

### Estoria

In [37]:
w2v_model.wv.most_similar(positive=["estoria"], topn=10)

[('e7', 0.38028454780578613),
 ('reliquary', 0.32181617617607117),
 ('roads', 0.3163324296474457),
 ('leyland', 0.2993161082267761),
 ('retired', 0.29784077405929565),
 ('arkelon', 0.28519758582115173),
 ('commerce', 0.2828056514263153),
 ('estorian', 0.28231507539749146),
 ('vintage', 0.28136008977890015),
 ('aresia', 0.2811938226222992)]

Estoria es una de las tres ciudades mas grandes y presentes en la historia. Sus habitantes son llamados estorian. Su economia se basa en el comercio, y su ubicacion mas iconica es un templo (denotado como zona E7) en el cual hay una reliquia (reliquary). Actualmente está en guerra con otra ciudad: Aresia.

### Vallus

In [38]:
w2v_model.wv.most_similar(positive=["vallus"], topn=10)

[('kyrah', 0.343248188495636),
 ('married', 0.3371690809726715),
 ('miracle', 0.31950101256370544),
 ('performing', 0.2918865978717804),
 ('volkan', 0.2873549163341522),
 ('goddess', 0.28729772567749023),
 ('sulla', 0.28409093618392944),
 ('argyn', 0.2812820076942444),
 ('cleric', 0.27932360768318176),
 ('bathhouse', 0.27904176712036133)]

Vallus (goddess), junto con otros 4 Dioses (Kyrah y Volkan siendo dos de ellos), son los deuteragonistas de la historia, ya que están del lado de los aventureros para luchar contra los Dioses antagonistas Sydon y Lutheria. Está casada (married) con el rey de una de las ciudades.

### Oracle

In [39]:
w2v_model.wv.most_similar(positive=["oracle"], topn=10)

[('versi', 0.4209427237510681),
 ('hook', 0.38196486234664917),
 ('prophesied', 0.35330653190612793),
 ('oracle’s', 0.3395397365093231),
 ('destiny', 0.3268973231315613),
 ('heleka', 0.3086612820625305),
 ('attendants', 0.302988201379776),
 ('fabled', 0.2922656536102295),
 ('boiling', 0.28950029611587524),
 ('encourages', 0.2870042026042938)]

El oraculo (Versi) es quien profetiza (prophesied) que el mundo está en peligro, y convoca a heroespara que hagan un juramento (destiny) y lo salven. Utiliza los vapores del agua hirviente (boiling) de un rio volcanico como medio para sus visiones. Cuando los aventureros la encuentran, está aprisionada por una bruja llamada Heleka. 

### Centaur

In [40]:
w2v_model.wv.most_similar(positive=["centaur"], topn=10)

[('druid', 0.42028653621673584),
 ('chieftain', 0.3463837206363678),
 ('hostages', 0.3432929813861847),
 ('coalition', 0.33578360080718994),
 ('scorpion', 0.29452553391456604),
 ('strongest', 0.29204317927360535),
 ('truce', 0.29186272621154785),
 ('warband', 0.28972527384757996),
 ("zakroth's", 0.28656506538391113),
 ('carcass', 0.2611949145793915)]

En esta historia, los centauros se agrupan en tribus lideradas por un capitan (chieftain), están aliados (coalition) con druidas (druid), se mueven en patrullas (warbands) para matar y mantener su territorio. 

### Dragon

In [41]:
w2v_model.wv.most_similar(positive=["dragon"], topn=10)

[('egg', 0.3522506654262543),
 ('brass', 0.3498973846435547),
 ('clone', 0.34094443917274475),
 ('eggs', 0.33431243896484375),
 ('wyrmling', 0.33144429326057434),
 ('e7', 0.3285750448703766),
 ('turtles', 0.3274625241756439),
 ('adult', 0.31750428676605225),
 ('hatch', 0.3130284547805786),
 ('huorath', 0.30567899346351624)]

Uno de los objetivos que deben cumplir los aventureros para salvar el mundo, es convertirse en Dragonlords. Para eso, requieren un dragon. Los dragones se encuentran en forma de huevo (egg), el cual deben cuidar hasta que se abra (hatch). Los dragones se clasifican por tipos (brass siendo uno de ellos) y por su edad, siendo Wyrmling y Adult dos de ellas.

### Hear

In [42]:
w2v_model.wv.most_similar(positive=["hear"], topn=10)

[('sounds', 0.36909207701683044),
 ('faint', 0.3643706440925598),
 ('whispering', 0.3575986325740814),
 ('beats', 0.3358883857727051),
 ('chorus', 0.32671448588371277),
 ('echoing', 0.29963380098342896),
 ('voices', 0.29205086827278137),
 ("isn't", 0.2824938893318176),
 ('inspire', 0.27686917781829834),
 ('farther', 0.27272072434425354)]

Hear es una palabra muy utilizada para describir a los aventureros lo que escuchan cuando investigan los distintos lugares del mundo. Vemos que todas las palabras estan relacionadas a adjetivos que describen sonidos.

### Trap

In [43]:
w2v_model.wv.most_similar(positive=["trap"], topn=10)

[('lever', 0.3447284698486328),
 ('trigger', 0.30654585361480713),
 ('triggers', 0.3064015209674835),
 ('mechanism', 0.28719663619995117),
 ('dimension', 0.28359702229499817),
 ('saddle', 0.27254432439804077),
 ('pulling', 0.2694096267223358),
 ('opening', 0.2643486261367798),
 ('outcropping', 0.2638585567474365),
 ('portcullis', 0.26384368538856506)]

Los aventureros se enfrentan a muchas trampas a lo largo de la historia, algunas relacionadas con palancas (lever), puertas (portcullis), u otros mecanismos (mechanism) que las desencadenan (trigger y triggers).

## Relaciones de vectores

Definimos algunos vectores para ver como se comportan cuando los relacionamos por operaciones aritmeticas como suma y resta.

In [50]:
vector1 = w2v_model.wv.get_vector("lutheria")
vector2 = w2v_model.wv.get_vector("sydon")
vector3 = w2v_model.wv.get_vector("kyrah")
vector4 = w2v_model.wv.get_vector("pythor")
vector5 = w2v_model.wv.get_vector("necropolis")
vector6 = w2v_model.wv.get_vector("damon")
print(w2v_model.wv.most_similar(vector1))
print(w2v_model.wv.most_similar(vector2))
print(w2v_model.wv.most_similar(vector3))
print(w2v_model.wv.most_similar(vector4))
print(w2v_model.wv.most_similar(vector5))
print(w2v_model.wv.most_similar(vector6))

[('lutheria', 1.0000001192092896), ('twin', 0.32119855284690857), ('dominion', 0.29322871565818787), ('mistress', 0.29278141260147095), ('humor', 0.29062914848327637), ('m20', 0.29052576422691345), ('sydon', 0.2856642007827759), ('confronting', 0.2844409644603729), ('scythe', 0.27126264572143555), ('pellenia', 0.2703191041946411)]
[('sydon', 1.0), ('demanded', 0.3137003779411316), ('visage', 0.30404555797576904), ('lutheria', 0.2856642007827759), ('prayer', 0.2799292504787445), ('obedience', 0.2785989046096802), ('enslaved', 0.27811679244041443), ('chalcia', 0.27806198596954346), ('valued', 0.2769514322280884), ('goloron', 0.2692423462867737)]
[('kyrah', 1.0), ("kyrah's", 0.38376134634017944), ('vallus', 0.3432481586933136), ('murphy', 0.3006424605846405), ('poet', 0.29154619574546814), ('music', 0.2902476489543915), ('pythor', 0.2825680673122406), ('recognizes', 0.2737751603126526), ('trickery', 0.2707512676715851), ('euria', 0.26841118931770325)]
[('pythor', 1.0), ('tortoise', 0.3413

Al sumar dos vectores, las palabras de cada vector van a ser las principales, por lo que las eliminamos para estudiar los terminos que quedan.

In [23]:
w2v_model.wv.most_similar(vector1 + vector2)[2:]

[('twin', 0.3413958251476288),
 ('m20', 0.32334211468696594),
 ('confronting', 0.319794237613678),
 ('enslaved', 0.31925731897354126),
 ('talieus', 0.3191412091255188),
 ('dominion', 0.3145463466644287),
 ('schemes', 0.31349533796310425),
 ('visage', 0.31207963824272156)]

Sumar el vector Lutheria y el vector Sydon nos da palabras como twin (son gemelos), m20 (zona donde se pueden encontrar a ambos personajes), schemes (ambos confabulan contra los aventureros).

In [24]:
w2v_model.wv.most_similar(vector3 + vector4)[2:]

[('vallus', 0.37169426679611206),
 ("kyrah's", 0.35637420415878296),
 ('volkan', 0.3387851119041443),
 ('married', 0.32192543148994446),
 ('murphy', 0.3085871934890747),
 ('miracle', 0.2962086498737335),
 ('ivory', 0.29618969559669495),
 ('onward', 0.2960524260997772)]

Sumar el vector Kyrah y Pythor (dos de los 5 dioses aliados) nos da palabras como Vallus (1 de los 5 Dioses), Volkan (otro de los 5 Dioses), miracle (todos los Dioses quieren el milagro de salvar el mundo), onward (todos los Dioses ayudan a que la historia se mueva hacia adelante).

In [51]:
w2v_model.wv.most_similar(vector5 + vector6)[2:]

[('telamok', 0.5626704096794128),
 ('lich', 0.44291505217552185),
 ('isadore', 0.43433693051338196),
 ('recognizes', 0.39297401905059814),
 ('laura', 0.3877762258052826),
 ('payment', 0.3783584535121918),
 ('manned', 0.36919304728507996),
 ('wizard', 0.3624146580696106)]

Sumar el vector necropolis y Damon nos da palabras como lich (Damon es un lich y se encuentra en la necropolis), Telamok (el nombre de la necropolis), wizard (antes de ser un lich, damon era un mago).

## Graficas

In [27]:
def reduce_dimensions(model, num_dimensions = 2 ):
     
    vectors = np.asarray(model.wv.vectors)
    labels = np.asarray(model.wv.index_to_key)  

    tsne = TSNE(n_components=num_dimensions, random_state=0)
    vectors = tsne.fit_transform(vectors)

    return vectors, labels

In [53]:
vecs, labels = reduce_dimensions(w2v_model)

START_INDEX = 1000
MAX_WORDS=200
fig = px.scatter(x=vecs[START_INDEX:START_INDEX + MAX_WORDS,0], y=vecs[START_INDEX:START_INDEX + MAX_WORDS,1], text=labels[START_INDEX:START_INDEX + MAX_WORDS])
fig.show()

In [52]:
vecs, labels = reduce_dimensions(w2v_model,3)

fig = px.scatter_3d(x=vecs[START_INDEX:START_INDEX + MAX_WORDS,0], y=vecs[START_INDEX:START_INDEX + MAX_WORDS,1], z=vecs[START_INDEX:START_INDEX + MAX_WORDS,2],text=labels[START_INDEX:START_INDEX + MAX_WORDS])
fig.update_traces(marker_size = 2)

Graficar 200 palabras de un vocabulario de 5000 palabras no resulta muy interesante. Y tratar de graficar mas palabras es lento (3 minutos en graficar 200 palabras). Mejor pasamos a la version web de projector tensorflow.

In [54]:
vectors = np.asarray(w2v_model.wv.vectors)
labels = list(w2v_model.wv.index_to_key)

np.savetxt("vectors.tsv", vectors, delimiter="\t")

with open("labels.tsv", "w") as fp:
    for item in labels:
        fp.write("%s\n" % item)

Utilizamos T-SNE (35 perplexity, 10 learning rate y 30 supervise) y a las 218 iteraciones encontramos algo interesante:

![T-SNE Iteracion 218](./images/images1.png)

En la zona roja tenemos palabras de uso muy recurrente como preposiciones (which, at, the, in, for, and, an) y verbos auxiliares (will, have, would, am).


![T-SNE Iteracion 218](./images/images2.png)

Como referencia, la zona anterior se circuló en rojo. La zona azul son palabras de muy poco uso (baker, bell, clayton). En la zona negra están todos los numeros:

 - Distancias de objetos
 - Denominaciones de zonas
 - Cantidades de dados (Dungeons & Dragons se basa en dados, y aqui se encuentran las notaciones de dados como 1d8 [1 dado de 8 caras], 1d20, d16, etc.)

![T-SNE Iteracion 350](./images/images3.png)

Luego de 350 iteraciones, el patron cambia significativamente. 

 - En la zona negra tenemos todo lo que son nombres propios (Williamson, Davis, Morgan, Zane, Theodore, Julien). 
 - En la zona azul tenemos nombres que aparecen en hechizos (animal de Speak with Animals, illusion de Minor Illusion, suggestion de Mass Suggestion, ray de Scorching Ray)
 - En la zona roja se mantiene la zona de numeros: todo lo relacionado a distancias, daño, dados.

Se dejó cargando hasta 600 iteraciones tratando de introducir perturbaciones ocasionalmente, pero las zonas anteriores se mantuvieron relativamente constantes, y los nucleos nuevos que se formaron eran relativamente pequeños.

![UMAP](./images/images4.png)

Se probó tambien con UMAP e, interesantemente, llegó a agrupaciones similares a las de T-SNE en las iteraciones tempranas:

 - En rojo están todas las preposiciones y verbos auxiliares o de mucho uso.
 - En azul están todos los nombres propios.
 